In [1]:
from requests import get
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('wordnet')

token = ''
domain = 'bodyfitsecret'


url = 'https://api.vk.com/method/wall.get'
# Определяем параметры первичного обращения к API ВКонтакте
params = {'access_token': token,
          'v':'5.130',
          'domain':domain,
          'count':1
         }
# Обращается к сообществу и извлекаем 1 запись со стены
inf_0 = get(url, params = params)
# Определяем количество записей на стене сообщества
posts_count = inf_0.json()['response']['count']
# Создаем списки для сохранения информации о записях со стены сообщества
date_list = []
text_list = []
# Формируем списко смещений для извлечения записей (пределе извлечения - 100 записей)
offsets_list = list(range(0,int((posts_count+1)*0.1),100))
# Устанавливает число извлекаемых за раз записей
params['count'] = 100
# Запускаем цикл перебора смещений
print ('Извлечение 10% записей со стены сообщества')
for offset in tqdm(offsets_list):
    # Устанавливаем смещение
    params['offset'] = offset
    # Образаемся к сообществу и извлекаем соответствующие 100 записей
    inf_0 = get(url, params = params)
    # Запускаем цикл перебора извлеченных записей
    for post in inf_0.json()['response']['items']:
        # Проверяем наличие у записи всех необходимых элементов информации
        if 'date' and 'text' in list(post.keys()):
            # Сохраняем дату записи в список
            date_list.append(datetime.fromtimestamp(post['date']).strftime('%d.%m.%Y'))
            # Сохраняем текст записи в список
            text_list.append(post['text'])
# Формируем единый структурированный массив            
vk_df = pd.DataFrame(list(zip(date_list, text_list)))
# Задаем наименования разделов единого структрированного массива
vk_df.columns = ['Дата записи', 'Текст записи']

stop_words = stopwords.words('russian')

noun = ['S-PRO', 'NUM=n', 'S', 'NONLEX']
verb = ['V']
adj = ['ANUM=pl', 'ANUM=n', 'ANUM=m', 'NUM=comp', 'A=comp2', 'A=m', 'A-PRO=f', 'A-PRO=pl', 'A-PRO', 'A=comp', 'A=sg', 'ADV=comp', 'A', 'A=pl', 'A=f', 'PRAEDIC=comp', 'A=n', 'A-PRO=n', 'ANUM=f', 'ADV', 'NUM=f', 'NUM=m']
# Список тегов вспомогательных частей речи: rec = ['A-PRO=m', 'NUM=acc', 'NUM=ciph', 'NUM=ins', 'NUM', 'NUM=gen', 'S-PRO=dat', 'PR', 'INTJ', 'NUM=nom', 'ADV=comp2', 'S-PRO=ins', 'ADV-PRO', 'S-PRO=acc', 'PARENTH', 'PRAEDIC', 'PART', 'CONJ']

text_tokens_list = []
text_tokens_count_list = []
text_tokens_list_full = []

print ('Токеннизация записей со стены сообщества')
for text in tqdm(vk_df['Текст записи']):
    text_1 = re.sub('[^\w\s]', '', text.lower())
    text_2 = re.sub('\n', ' ', text_1)
    if text_2.strip() != '':
        text_3 = word_tokenize(text_2.strip())
        text_4 = pos_tag(text_3, lang = 'rus')
        text_5 = []
        for text_tag in text_4:
            if text_tag[1] in noun:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'n'))
            elif text_tag[1] in verb:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'v'))
            elif text_tag[1] in adj:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'a'))
        text_6 = []
        for token in text_5:
            if token not in stop_words:
                text_6.append(token)
        if len(text_6) > 0:
            text_tokens_list.append(text_6)
            text_tokens_list_full.extend(text_6)
            text_tokens_count_list.append(len(text_6))
        else:
            text_tokens_list.append([])
            text_tokens_count_list.append(0)
    else:
        text_tokens_list.append([])
        text_tokens_count_list.append(0)
        
vk_df['Токены записи'] = text_tokens_list
vk_df['Длина списка токенов'] = text_tokens_count_list

vk_df = vk_df[vk_df['Длина списка токенов'] > 0]

tokens_count_list = Counter(text_tokens_list_full)

main_tokens_df = pd.DataFrame(list(zip(tokens_count_list.keys(), tokens_count_list.values())))

main_tokens_df.columns = ['Токен', 'Частота встречаемости']

main_tokens_df_cor = main_tokens_df[main_tokens_df['Частота встречаемости'] > np.array(main_tokens_df['Частота встречаемости']).mean()]

main_tokens_df_cor

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Извлечение 10% записей со стены сообщества



Токеннизация записей со стены сообщества


,Токен,Частота встречаемости
0,отличная,66
1,йога,23
2,животика,36
3,будь,6
4,форме,19
...,...,...
12367,банку,6
12368,овсянку,5
12369,закрываем,6
12380,отрубей,5


In [2]:
from requests import get
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import numpy as np

token = ''

# Определяем url обращения к API ВКонтакте
url = 'https://api.vk.com/method/wall.get'
# Определяем параметры первичного обращения к API ВКонтакте
params = {'access_token': token,
          'v':'5.130',
          'domain':domain,
          'count':1
         }
# Обращается к сообществу и извлекаем 1 запись со стены
inf_0 = get(url, params = params)
# Определяем количество записей на стене сообщества
print(inf_0.json())
posts_count = inf_0.json()['response']['count']
# Создаем списки для сохранения информации о записях со стены сообщества
date_list = []
text_list = []
# Формируем списко смещений для извлечения записей (пределе извлечения - 100 записей)
offsets_list = list(range(0,(posts_count+1),100))
# Устанавливает число извлекаемых за раз записей
params['count'] = 100
# Запускаем цикл перебора смещений
print ('Извлечение записей со стены сообщества')
for offset in tqdm(offsets_list):
    # Устанавливаем смещение
    params['offset'] = offset
    # Образаемся к сообществу и извлекаем соответствующие 100 записей
    inf_0 = get(url, params = params)
    # Запускаем цикл перебора извлеченных записей
    for post in inf_0.json()['response']['items']:
        # Проверяем наличие у записи всех необходимых элементов информации
        if 'date' and 'text' in list(post.keys()):
            # Сохраняем дату записи в список
            date_list.append(datetime.fromtimestamp(post['date']).strftime('%d.%m.%Y'))
            # Сохраняем текст записи в список
            text_list.append(post['text'])
# Формируем единый структурированный массив            
vk_term_df = pd.DataFrame(list(zip(date_list, text_list)))
# Задаем наименования разделов единого структрированного массива
vk_term_df.columns = ['Дата записи', 'Текст записи']

stop_words = stopwords.words('russian')

noun = ['S-PRO', 'NUM=n', 'S', 'NONLEX']
verb = ['V']
adj = ['ANUM=pl', 'ANUM=n', 'ANUM=m', 'NUM=comp', 'A=comp2', 'A=m', 'A-PRO=f', 'A-PRO=pl', 'A-PRO', 'A=comp', 'A=sg', 'ADV=comp', 'A', 'A=pl', 'A=f', 'PRAEDIC=comp', 'A=n', 'A-PRO=n', 'ANUM=f', 'ADV', 'NUM=f', 'NUM=m']
# Список тегов вспомогательных частей речи: rec = ['A-PRO=m', 'NUM=acc', 'NUM=ciph', 'NUM=ins', 'NUM', 'NUM=gen', 'S-PRO=dat', 'PR', 'INTJ', 'NUM=nom', 'ADV=comp2', 'S-PRO=ins', 'ADV-PRO', 'S-PRO=acc', 'PARENTH', 'PRAEDIC', 'PART', 'CONJ']

text_tokens_list_full = []

print ('Токеннизация записей со стены сообщества')
for text in tqdm(vk_term_df['Текст записи']):
    text_1 = re.sub('[^\w\s]', '', text.lower())
    text_2 = re.sub('\n', ' ', text_1)
    if text_2.strip() != '':
        text_3 = word_tokenize(text_2.strip())
        text_4 = pos_tag(text_3, lang = 'rus')
        text_5 = []
        for text_tag in text_4:
            if text_tag[1] in noun:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'n'))
            elif text_tag[1] in verb:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'v'))
            elif text_tag[1] in adj:
                text_5.append(WordNetLemmatizer().lemmatize(text_tag[0], pos = 'a'))
        text_6 = []
        for token in text_5:
            if token not in stop_words:
                text_6.append(token)
        if len(text_6) > 0:
            text_tokens_list_full.extend(text_6)

tokens_count_list = Counter(text_tokens_list_full)

term_tokens_df = pd.DataFrame(list(zip(tokens_count_list.keys(), tokens_count_list.values())))

term_tokens_df.columns = ['Токен', 'Частота встречаемости']

term_tokens_df_cor = term_tokens_df[term_tokens_df['Частота встречаемости'] > np.array(term_tokens_df['Частота встречаемости']).mean()]

term_tokens_df_cor

{'response': {'count': 48837, 'items': [{'id': 224180, 'from_id': -61040032, 'owner_id': -61040032, 'date': 1619982780, 'marked_as_ads': 0, 'post_type': 'post', 'text': 'Отличная йога для вашего животика! Будь в форме!', 'is_pinned': 1, 'attachments': [{'type': 'video', 'video': {'access_key': '155acba4ee807798b2', 'can_comment': 0, 'can_like': 1, 'can_repost': 1, 'can_subscribe': 1, 'can_add_to_faves': 1, 'can_add': 1, 'comments': 0, 'date': 1619948610, 'description': '', 'duration': 157, 'image': [{'height': 96, 'url': 'https://i.mycdn.me/getVideoPreview?id=1094054185665&idx=0&type=39&tkn=6qdcHCGIoFxTjtaSf9FpAHmyHz4&fn=vid_s', 'width': 130, 'with_padding': 1}, {'height': 120, 'url': 'https://i.mycdn.me/getVideoPreview?id=1094054185665&idx=0&type=39&tkn=6qdcHCGIoFxTjtaSf9FpAHmyHz4&fn=vid_m', 'width': 160, 'with_padding': 1}, {'height': 240, 'url': 'https://i.mycdn.me/getVideoPreview?id=1094054185665&idx=0&type=39&tkn=6qdcHCGIoFxTjtaSf9FpAHmyHz4&fn=vid_l', 'width': 320, 'with_padding':


Токеннизация записей со стены сообщества


,Токен,Частота встречаемости
0,отличная,390
1,йога,853
2,животика,442
3,будь,340
4,форме,855
...,...,...
92225,сooбществe,47
98385,познайте,58
103935,club61040032получайте,55
105606,сбн,42


In [3]:
crossing_tokens_list = list(set(main_tokens_df_cor['Токен']) & set(term_tokens_df_cor['Токен']))

# Квантификатор 1 - длина списка общих токенов
k_1 = len(crossing_tokens_list)

# Квантификатор 2 - доля общих токенов в массиве токенов, описывающих потребителя
k_2 = len(crossing_tokens_list) / len(list(main_tokens_df_cor['Токен']))

# Квантификатор 3 - доля общих токенов в массиве токенов, описывающих тематическую компоненты
k_3 = len(crossing_tokens_list) / len(list(term_tokens_df_cor['Токен']))

# Квантификатор 4 - число общих токенов, скорректирвоанное на удельный вес
crossing_tokens_imp_list = []
for token in crossing_tokens_list:
    crossing_tokens_imp_list.append(term_tokens_df_cor[term_tokens_df_cor['Токен'] == token].iloc[0]['Частота встречаемости'])
k_4 = sum(crossing_tokens_imp_list)

# Квантификатор 5 - доля общих токенов в массиве токенов, описывающих потребителя, скорректированная на удельный вес
crossing_tokens_imp_list = []
for token in crossing_tokens_list:
    crossing_tokens_imp_list.append(main_tokens_df_cor[main_tokens_df_cor['Токен'] == token].iloc[0]['Частота встречаемости'])
k_5 = sum(crossing_tokens_imp_list) / sum(list(main_tokens_df_cor['Частота встречаемости']))

# Квантификатор 6 - доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес
crossing_tokens_imp_list = []
for token in crossing_tokens_list:
    crossing_tokens_imp_list.append(term_tokens_df_cor[term_tokens_df_cor['Токен'] == token].iloc[0]['Частота встречаемости'])
k_6 = sum(crossing_tokens_imp_list) / sum(list(term_tokens_df_cor['Частота встречаемости']))

k_basic_df = pd.DataFrame([k_1, k_2, k_3, k_4, k_5, k_6], columns = ['Значения квантификаторов'])

k_basic_df.index = ['Длина списка общих токенов',
                    'Доля общих токенов в массиве токенов, описывающих потребителя',
                    'Доля общих токенов в массиве токенов, описывающих тематическую компоненту',
                    'Число общих токенов, скорректирвоанное на удельный вес',
                    'Доля общих токенов в массиве токенов, описывающих потребителя, скорректированная на удельный вес',
                    'Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес'
                   ]

k_basic_df

,Значения квантификаторов
Длина списка общих токенов,2.173000e+03
"Доля общих токенов в массиве токенов, описывающих потребителя",8.865769e-01
"Доля общих токенов в массиве токенов, описывающих тематическую компоненту",1.636665e-01
"Число общих токенов, скорректирвоанное на удельный вес",1.926557e+06
"Доля общих токенов в массиве токенов, описывающих потребителя, скорректированная на удельный вес",9.539475e-01
"Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес",5.435413e-01


In [4]:
vk_df_1 = vk_df.reset_index(drop=True)

list_of_crossing_tokens_lists = []
for tokens_list in vk_df['Токены записи']:
    list_of_crossing_tokens_lists.append(list(set(tokens_list) & set(term_tokens_df_cor['Токен'])))
                                                                                
# Квантификатор 1 - длина списка общих токенов в записи
k_1_list = []
for crossing_tokens_list in list_of_crossing_tokens_lists:
    k_1 = len(crossing_tokens_list)
    k_1_list.append(k_1)
    
# Квантификатор 2 - доля общих токенов в массиве токенов, описывающих запись
k_2_list = []
num_of_post = 0
for crossing_tokens_list in list_of_crossing_tokens_lists:
    k_2 = len(crossing_tokens_list) / vk_df_1['Длина списка токенов'][num_of_post]
    k_2_list.append(k_2)
    num_of_post += 1
    
# Квантификатор 3 - доля общих токенов (описывающих запись) в массиве токенов, описывающих тематическую компоненты
k_3_list = []
num_of_post = 0
for crossing_tokens_list in list_of_crossing_tokens_lists:
    k_3 = len(crossing_tokens_list) / len(list(term_tokens_df_cor['Токен']))
    k_3_list.append(k_3)
    num_of_post += 1

vk_df_1['Длина списка общих токенов в записи'] = k_1_list
vk_df_1['Доля общих токенов в массиве токенов, описывающих запись'] = k_2_list
vk_df_1['Доля общих токенов (описывающих запись) в массиве токенов, описывающих тематическую компоненты'] = k_3_list

vk_df_1

,Дата записи,Текст записи,Токены записи,Длина списка токенов,Длина списка общих токенов в записи,"Доля общих токенов в массиве токенов, описывающих запись","Доля общих токенов (описывающих запись) в массиве токенов, описывающих тематическую компоненты"
0,02.05.2021,Отличная йога для вашего животика! Будь в форме!,"[отличная, йога, животика, будь, форме]",5,5,1.0000,0.000377
1,20.07.2021,Тренировка на все тело🔥,"[тренировка, тело]",2,2,1.0000,0.000151
2,20.07.2021,Убираем жир между ног👌🏻,"[убираем, жир, ног]",3,3,1.0000,0.000226
3,20.07.2021,Для шикарных ягодиц😍,"[шикарных, ягодиц]",2,1,0.5000,0.000075
4,20.07.2021,Подтягиваем ягодицы🔥,"[подтягиваем, ягодицы]",2,2,1.0000,0.000151
...,...,...,...,...,...,...,...
4660,15.05.2020,Вариант тренировки для пресса! \n \nНа видео п...,"[вариант, тренировки, пресса, видео, показано,...",32,30,0.9375,0.002260
4661,14.05.2020,"Часто болит поясница?\n Или делаешь планку, а ...","[часто, болит, поясница, делаешь, планку, чувс...",8,8,1.0000,0.000603
4662,14.05.2020,Будь в форме,"[будь, форме]",2,2,1.0000,0.000151
4663,14.05.2020,Главное - желание! А поддержать мышцы в отличн...,"[главное, желание, поддержать, мышцы, отличной...",8,7,0.8750,0.000527


In [5]:
!pip install dostoevsky
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
!python -m dostoevsky download fasttext-social-network-model

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

negative_level_list = []
positive_level_list = []
neutral_level_list = []
speech_level_list = []            
print ('Оценка тональности записей')
for post in tqdm(vk_df['Текст записи']):
    if len(post.strip()) > 5:
        tonal_analis_list = model.predict([post])
        neutral_level_list.append(tonal_analis_list[0]['neutral'])
        negative_level_list.append(tonal_analis_list[0]['negative'])
        positive_level_list.append(tonal_analis_list[0]['positive'])
        speech_level_list.append(tonal_analis_list[0]['speech'])
    else:
        negative_level_list.append(0)
        positive_level_list.append(0)
        neutral_level_list.append(0)
        speech_level_list.append(0)
        
tonal_disp_level_list = []
y = 0
for x in negative_level_list:
    if x > 0:
        tonal_disp_level_list.append(positive_level_list[y] / x)
    else:
        tonal_disp_level_list.append(0)
    y += 1
        
tonal_full_level_list = []
y = 0
for x in neutral_level_list:
    if x > 0:
        tonal_full_level_list.append((positive_level_list[y] + negative_level_list[y]) / x)
    else:
        tonal_full_level_list.append(0)
    y += 1
    
vk_df['Уровень нейтральности'] = neutral_level_list
vk_df['Уровень позитивности'] = positive_level_list
vk_df['Уровень негативности'] = negative_level_list
vk_df['Уровень разговорности'] = speech_level_list
vk_df['Уровень эмоционального разрыва'] = tonal_disp_level_list
vk_df['Уровень общей эмоциональности'] = tonal_full_level_list

vk_df_final = vk_df[(vk_df['Уровень нейтральности'] != 0) & (vk_df['Уровень позитивности'] != 0) & (vk_df['Уровень негативности'] != 0) & (vk_df['Уровень разговорности'] != 0) & (vk_df['Уровень эмоционального разрыва'] != 0) & (vk_df['Уровень общей эмоциональности'] != 0)]

vk_df_final.describe()

     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.7.0-py2.py3-none-any.whl (199 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095737 sha256=8a95f5fd0472cc601a248635f11c085659f59d3cea3ef7802989feb6f1a1cd15
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
Оценка тональности записей


,Длина списка токенов,Уровень нейтральности,Уровень позитивности,Уровень негативности,Уровень разговорности,Уровень эмоционального разрыва,Уровень общей эмоциональности
count,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000
mean,12.399314,0.692595,0.125698,0.071291,0.009811,1054.270117,198.315508
std,30.852302,0.291974,0.212019,0.091652,0.054957,9221.002478,4350.958392
min,1.000000,0.000010,0.000010,0.000010,0.000010,0.000023,0.000020
25%,2.000000,0.507822,0.007826,0.010338,0.000010,0.213141,0.052488
50%,4.000000,0.782673,0.042098,0.038476,0.000840,0.804465,0.169178
75%,7.000000,0.941664,0.125933,0.095359,0.005395,4.200249,0.490201
max,412.000000,1.000010,1.000010,0.859674,0.998726,100000.967504,100207.041763


In [6]:
!pip install xlsxwriter
import xlsxwriter
path = '/content/bodyfitsecret_Analysis.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
main_tokens_df_cor.to_excel(writer, sheet_name = 'tokens')
term_tokens_df_cor.to_excel(writer, sheet_name = 'terms') 
vk_df_1.to_excel(writer, sheet_name = 'posts_split')
vk_df_final.to_excel(writer, sheet_name = 'tonal')
writer.save()
writer.close()

     |████████████████████████████████| 149 kB 4.8 MB/s 


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
